In [1]:
from fr_models.config import *

print("---------------------fr模型----------------------")
fr_model_names = list(threshold_lfw.keys())
# fr_model_names = ['testirse50', 'testir152', 'testmobileface', 'testfacenet']
fr_model_names = ['IR50-PGDSoftmax','IR50-TradesCosFace']

---------------------fr模型----------------------


In [2]:
import torch
import torchvision
import torch.nn.functional as F
from utils.dataset import Dataset
from fr_models.get_model import getmodel
from tqdm import tqdm

def loadDataset(dataset_dir, mean, std, mode, batch_size):
    dataset = Dataset(dataset_dir, mode)
    transforms = torchvision.transforms.Compose([
        torchvision.transforms.ToTensor(),  # 图片转为Tensor [0,1]
        torchvision.transforms.Normalize(mean=mean, std=std),  # 归一化
    ])
    dataset.start_batch_queue(
        batch_size=batch_size,
        batch_format="random_samples",
        transforms=transforms,
        num_threads=2,
        untarget_same=False
    )
    return dataset

def evalFRModel(model, img_shape, mode, dataset):
    epoch_size = 6000 // batch_size
    simi_scores = []
    for _ in tqdm(range(0, epoch_size), total=epoch_size):
        batch = dataset.pop_batch_queue()
        with torch.no_grad():
            # 获取对应的批次数据
            source_faces = batch['sources'].cuda()
            target_faces = batch['targets'].cuda()
            emb_source_faces = model.forward(F.interpolate((source_faces*0.5+0.5)*255, size=img_shape, mode='bilinear'))
            emb_target_faces = model.forward(F.interpolate((target_faces*0.5+0.5)*255, size=img_shape, mode='bilinear'))
            batch_scores = torch.cosine_similarity(emb_source_faces, emb_target_faces)
            simi_scores.extend(batch_scores.tolist())
    th = threshold_lfw[model_name]['cos']
    print("识别阈值:", th)
    mean_simi = sum(simi_scores)/len(simi_scores)
    print("平均相似分数:", mean_simi)
    success_rate = sum(score > th if mode=="untarget" else score < th for score in simi_scores) / len(simi_scores)
    print(model_name, "benchmark rate:%f" % threshold_lfw[model_name]['cos_acc'])
    print(model_name, "success rate:%f" % success_rate)
    return th, mean_simi, threshold_lfw[model_name]['cos_acc'], success_rate
    

In [3]:
from fr_models.get_model import getmodel

batch_size=32
dataset_dir="D:/datasets/lfw/lfw-aligned-112x112"
untarget_dataset = loadDataset(dataset_dir=dataset_dir,
                                   mean=[0.5, 0.5, 0.5],
                                   std=[0.5, 0.5, 0.5],
                                   mode="untarget",
                                   batch_size=batch_size)
target_dataset = loadDataset(dataset_dir=dataset_dir,
                                   mean=[0.5, 0.5, 0.5],
                                   std=[0.5, 0.5, 0.5],
                                   mode="target",
                                   batch_size=batch_size)
benchmark = {}
print("使用数据集 %s:" % dataset_dir)
for model_name in fr_model_names:
    with torch.no_grad():
        benchmark[model_name]={}
        print("评估%s中:" % (model_name))
        model, img_shape = getmodel(model_name)
        print("    TAR:")
        benchmark[model_name]['TAR']=evalFRModel(model, img_shape, "untarget", untarget_dataset)
        print("    FAR:")
        benchmark[model_name]['FAR']=evalFRModel(model, img_shape, "target", target_dataset)
import json
print(json.dumps(benchmark, indent=2))
'''
{
  "ShuffleNet_V2_GDConv-stride1": {
    "TAR": [
      0.22403636574745178,
      0.5545654333306148,
      0.9915,
      0.9856283422459893
    ],
    "FAR": [
      0.22403636574745178,
      0.035012854338475105,
      0.9915,
      0.9956550802139037
    ]
  },
  "Mobilenet": {
    "TAR": [
      0.15846577286720276,
      0.5271046995350289,
      0.9935,
      0.9883021390374331
    ],
    "FAR": [
      0.15846577286720276,
      0.00574880088655369,
      0.9935,
      0.9953208556149733
    ]
  },
  "MobilenetV2-stride1": {
    "TAR": [
      0.24340271949768066,
      0.5736437767481352,
      0.9905,
      0.9879679144385026
    ],
    "FAR": [
      0.24340271949768066,
      0.029356853724751473,
      0.9905,
      0.9939839572192514
    ]
  },
  "CosFace": {
    "TAR": [
      0.24625787138938904,
      0.5870342402420764,
      0.9868333333333333,
      0.982620320855615
    ],
    "FAR": [
      0.24625787138938904,
      0.0388618732872528,
      0.9868333333333333,
      0.982620320855615
    ]
  },
  "FaceNet-VGGFace2": {
    "TAR": [
      0.42074376344680786,
      0.7559484232822185,
      0.9923333333333333,
      0.9893048128342246
    ],
    "FAR": [
      0.42074376344680786,
      0.035508966864877936,
      0.9923333333333333,
      0.9895719251336899
    ]
  },
  "Mobilenet-stride1": {
    "TAR": [
      0.19342336058616638,
      0.5710021915610874,
      0.9956666666666667,
      0.9919786096256684
    ],
    "FAR": [
      0.19342336058616638,
      0.008473093308955692,
      0.9956666666666667,
      0.9971590909090909
    ]
  },
  "SphereFace": {
    "TAR": [
      0.34931832551956177,
      0.6346463531873283,
      0.9818333333333333,
      0.9751002673796791
    ],
    "FAR": [
      0.34931832551956177,
      0.09115480367472628,
      0.9818333333333333,
      0.9807820855614974
    ]
  },
  "MobileFace": {
    "TAR": [
      0.21116749942302704,
      0.5738695791809207,
      0.9945,
      0.9914772727272727
    ],
    "FAR": [
      0.21116749942302704,
      0.009152118811879698,
      0.9945,
      0.9963235294117647
    ]
  },
  "FaceNet-casia": {
    "TAR": [
      0.4289606213569641,
      0.7107233562417089,
      0.981,
      0.972927807486631
    ],
    "FAR": [
      0.4289606213569641,
      0.1309350894319026,
      0.981,
      0.9794451871657754
    ]
  },
  "MobilenetV2": {
    "TAR": [
      0.22666725516319275,
      0.5640347774564176,
      0.9933333333333333,
      0.9841243315508021
    ],
    "FAR": [
      0.22666725516319275,
      0.007508023444035235,
      0.9933333333333333,
      0.9953208556149733
    ]
  },
  "ArcFace": {
    "TAR": [
      0.28402677178382874,
      0.633667108417927,
      0.995,
      0.9891377005347594
    ],
    "FAR": [
      0.28402677178382874,
      0.0140841106766988,
      0.995,
      0.9978275401069518
    ]
  },
  "ResNet50": {
    "TAR": [
      0.19116485118865967,
      0.6031841859433122,
      0.9971666666666666,
      0.9938168449197861
    ],
    "FAR": [
      0.19116485118865967,
      0.003980067884020338,
      0.9971666666666666,
      0.9981617647058824
    ]
  },
  "ShuffleNet_V1_GDConv": {
    "TAR": [
      0.19073212146759033,
      0.5581226770873157,
      0.9946666666666667,
      0.9904745989304813
    ],
    "FAR": [
      0.19073212146759033,
      0.011514300876538291,
      0.9946666666666667,
      0.9953208556149733
    ]
  },
  "ResNet50-casia": {
    "TAR": [
      0.1854616403579712,
      0.5236097511327713,
      0.986,
      0.9829545454545454
    ],
    "FAR": [
      0.1854616403579712,
      0.020249963900857832,
      0.986,
      0.9762700534759359
    ]
  },
  "IR50-Am": {
    "TAR": [
      0.45836973190307617,
      0.686318062992896,
      0.9921666666666666,
      0.9899732620320856
    ],
    "FAR": [
      0.45836973190307617,
      0.3194407504861228,
      0.9921666666666666,
      0.9913101604278075
    ]
  },
  "IR50-ArcFace": {
    "TAR": [
      0.27737492322921753,
      0.678309206876351,
      0.997,
      0.9936497326203209
    ],
    "FAR": [
      0.27737492322921753,
      0.00801246330797402,
      0.997,
      0.9991644385026738
    ]
  },
  "IR50-CosFace": {
    "TAR": [
      0.22401964664459229,
      0.6294404039885301,
      0.997,
      0.9938168449197861
    ],
    "FAR": [
      0.22401964664459229,
      0.00664144790894632,
      0.997,
      0.9986631016042781
    ]
  },
  "IR50-Softmax": {
    "TAR": [
      0.34233152866363525,
      0.7387041095213125,
      0.996,
      0.9936497326203209
    ],
    "FAR": [
      0.34233152866363525,
      0.03411596298114587,
      0.996,
      0.9954879679144385
    ]
  },
  "IR50-SphereFace": {
    "TAR": [
      0.361619234085083,
      0.7571889617241282,
      0.9958333333333333,
      0.9941510695187166
    ],
    "FAR": [
      0.361619234085083,
      0.012270002187963359,
      0.9958333333333333,
      0.9956550802139037
    ]
  },
  "IR50-PGDSoftmax": {
    "TAR": [
      0.3052944242954254,
      0.5224100640166568,
      0.913,
      0.8913770053475936
    ],
    "FAR": [
      0.3052944242954254,
      0.13586034558705343,
      0.913,
      0.9286430481283422
    ]
  },
  "IR50-TradesSoftmax": {
    "TAR": [
      0.32782697677612305,
      0.5474989166592851,
      0.9095,
      0.8920454545454546
    ],
    "FAR": [
      0.32782697677612305,
      0.15139548837701866,
      0.9095,
      0.9266377005347594
    ]
  },
  "IR50-PGDArcFace": {
    "TAR": [
      0.37509775161743164,
      0.5403258596960315,
      0.8766666666666667,
      0.8288770053475936
    ],
    "FAR": [
      0.37509775161743164,
      0.21964961855802567,
      0.8766666666666667,
      0.8997326203208557
    ]
  },
  "IR50-TradesArcFace": {
    "TAR": [
      0.852626383304596,
      0.9300553913422447,
      0.9506666666666667,
      0.9667446524064172
    ],
    "FAR": [
      0.852626383304596,
      0.7430060774107827,
      0.9506666666666667,
      0.9314839572192514
    ]
  },
  "IR50-PGDCosFace": {
    "TAR": [
      0.2336357682943344,
      0.39558989097838,
      0.8618333333333333,
      0.8295454545454546
    ],
    "FAR": [
      0.2336357682943344,
      0.13054349993801934,
      0.8618333333333333,
      0.8751671122994652
    ]
  },
  "IR50-TradesCosFace": {
    "TAR": [
      0.6367585062980652,
      0.7639066341487482,
      0.9073333333333333,
      0.8846925133689839
    ],
    "FAR": [
      0.6367585062980652,
      0.4596707873428639,
      0.9073333333333333,
      0.9296457219251337
    ]
  },
  "IR50-PGDAm": {
    "TAR": [
      0.415134072303772,
      0.5527289948540735,
      0.85,
      0.8136697860962567
    ],
    "FAR": [
      0.415134072303772,
      0.32055265119767445,
      0.85,
      0.8713235294117647
    ]
  },
  "IR50-PGDSphereFace": {
    "TAR": [
      0.5209366679191589,
      0.7586736102698354,
      0.8881666666666667,
      0.893048128342246
    ],
    "FAR": [
      0.5209366679191589,
      0.14576010059845138,
      0.8881666666666667,
      0.8688168449197861
    ]
  },
  "CASIA-Am": {
    "TAR": [
      0.4614875912666321,
      0.6570319125757498,
      0.978,
      0.9632352941176471
    ],
    "FAR": [
      0.4614875912666321,
      0.31093917002174304,
      0.978,
      0.9837901069518716
    ]
  },
  "CASIA-ArcFace": {
    "TAR": [
      0.3242388069629669,
      0.6606621580138643,
      0.9905,
      0.984792780748663
    ],
    "FAR": [
      0.3242388069629669,
      0.032969968659348196,
      0.9905,
      0.9899732620320856
    ]
  },
  "CASIA-CosFace": {
    "TAR": [
      0.23147112131118774,
      0.57475772034657,
      0.9911666666666666,
      0.9914772727272727
    ],
    "FAR": [
      0.23147112131118774,
      0.019781857927782427,
      0.9911666666666666,
      0.9899732620320856
    ]
  },
  "CASIA-Softmax": {
    "TAR": [
      0.3241730332374573,
      0.6462740708742649,
      0.9883333333333333,
      0.9857954545454546
    ],
    "FAR": [
      0.3241730332374573,
      0.04384005715596467,
      0.9883333333333333,
      0.986129679144385
    ]
  },
  "CASIA-SphereFace": {
    "TAR": [
      0.4365990459918976,
      0.7471717953221887,
      0.9765,
      0.9727606951871658
    ],
    "FAR": [
      0.4365990459918976,
      0.05307389100470996,
      0.9765,
      0.9717580213903744
    ]
  },
  "IR50-Softmax-BR": {
    "TAR": [
      0.33871138095855713,
      0.7380299227073152,
      0.996,
      0.9943181818181818
    ],
    "FAR": [
      0.33871138095855713,
      0.03434138125578131,
      0.996,
      0.9944852941176471
    ]
  },
  "IR50-Softmax-JPEG": {
    "TAR": [
      0.34150177240371704,
      0.7373340896350695,
      0.9958333333333333,
      0.99298128342246
    ],
    "FAR": [
      0.34150177240371704,
      0.0344092214981889,
      0.9958333333333333,
      0.9956550802139037
    ]
  },
  "IR50-Softmax-RP": {
    "TAR": [
      0.35351189970970154,
      0.7394883740994641,
      0.9941666666666666,
      0.9934826203208557
    ],
    "FAR": [
      0.35351189970970154,
      0.04619174555339664,
      0.9941666666666666,
      0.995153743315508
    ]
  }
}
'''

13233 images of 5749 classes loaded
13233 images of 5749 classes loaded
使用数据集 D:/datasets/lfw/lfw-aligned-112x112:
评估testirse50中:
    TAR:


  0%|          | 0/187 [01:00<?, ?it/s]


Empty: 

In [3]:
from fr_models.get_model import getmodel

batch_size=32
dataset_dir="/home/com/yy/datasets/CASIA-WebFace/casia-aligned-112x112/"
untarget_dataset = loadDataset(dataset_dir=dataset_dir,
                                   mean=[0.5, 0.5, 0.5],
                                   std=[0.5, 0.5, 0.5],
                                   mode="untarget",
                                   batch_size=batch_size)
target_dataset = loadDataset(dataset_dir=dataset_dir,
                                   mean=[0.5, 0.5, 0.5],
                                   std=[0.5, 0.5, 0.5],
                                   mode="target",
                                   batch_size=batch_size)
benchmark = {}
print("使用数据集 %s:" % dataset_dir)
for model_name in fr_model_names:
    with torch.no_grad():
        benchmark[model_name]={}
        print("评估%s中:" % (model_name))
        model, img_shape = getmodel(model_name)
        print("    TAR:")
        benchmark[model_name]['TAR']=evalFRModel(model, img_shape, "untarget", untarget_dataset)
        print("    FAR:")
        benchmark[model_name]['FAR']=evalFRModel(model, img_shape, "target", target_dataset)
import json
print(json.dumps(benchmark, indent=2))
'''
{
  "ShuffleNet_V2_GDConv-stride1": {
    "TAR": [
      0.22403636574745178,
      0.38950889972352953,
      0.9915,
      0.8079879679144385
    ],
    "FAR": [
      0.22403636574745178,
      0.03514139656262923,
      0.9915,
      0.99298128342246
    ]
  },
  "Mobilenet": {
    "TAR": [
      0.15846577286720276,
      0.33970919899009966,
      0.9935,
      0.8153409090909091
    ],
    "FAR": [
      0.15846577286720276,
      0.007424565412303032,
      0.9935,
      0.9944852941176471
    ]
  },
  "MobilenetV2-stride1": {
    "TAR": [
      0.24340271949768066,
      0.4057902479143521,
      0.9905,
      0.797292780748663
    ],
    "FAR": [
      0.24340271949768066,
      0.03793284978065764,
      0.9905,
      0.9914772727272727
    ]
  },
  "CosFace": {
    "TAR": [
      0.24625787138938904,
      0.4757590359064407,
      0.9868333333333333,
      0.8492647058823529
    ],
    "FAR": [
      0.24625787138938904,
      0.02677795590409034,
      0.9868333333333333,
      0.991644385026738
    ]
  },
  "FaceNet-VGGFace2": {
    "TAR": [
      0.42074376344680786,
      0.5777556795582056,
      0.9923333333333333,
      0.8051470588235294
    ],
    "FAR": [
      0.42074376344680786,
      0.015297117367778085,
      0.9923333333333333,
      0.9948195187165776
    ]
  },
  "Mobilenet-stride1": {
    "TAR": [
      0.19342336058616638,
      0.40369698638633394,
      0.9956666666666667,
      0.8559491978609626
    ],
    "FAR": [
      0.19342336058616638,
      0.010004957837765765,
      0.9956666666666667,
      0.9973262032085561
    ]
  },
  "SphereFace": {
    "TAR": [
      0.34931832551956177,
      0.4916775505866838,
      0.9818333333333333,
      0.7799131016042781
    ],
    "FAR": [
      0.34931832551956177,
      0.05791943540088808,
      0.9818333333333333,
      0.9923128342245989
    ]
  },
  "MobileFace": {
    "TAR": [
      0.21116749942302704,
      0.40472915966241474,
      0.9945,
      0.8460895721925134
    ],
    "FAR": [
      0.21116749942302704,
      0.011561774252750475,
      0.9945,
      0.9961564171122995
    ]
  },
  "FaceNet-casia": {
    "TAR": [
      0.4289606213569641,
      0.5683028281381214,
      0.981,
      0.7829211229946524
    ],
    "FAR": [
      0.4289606213569641,
      0.0940956809953444,
      0.981,
      0.9906417112299465
    ]
  },
  "MobilenetV2": {
    "TAR": [
      0.22666725516319275,
      0.3839497219179421,
      0.9933333333333333,
      0.7876002673796791
    ],
    "FAR": [
      0.22666725516319275,
      0.011135533766651072,
      0.9933333333333333,
      0.9964906417112299
    ]
  },
  "ArcFace": {
    "TAR": [
      0.28402677178382874,
      0.46903507017901225,
      0.995,
      0.838903743315508
    ],
    "FAR": [
      0.28402677178382874,
      0.015232160427107384,
      0.995,
      0.9984959893048129
    ]
  },
  "ResNet50": {
    "TAR": [
      0.19116485118865967,
      0.4209562875198158,
      0.9971666666666666,
      0.8633021390374331
    ],
    "FAR": [
      0.19116485118865967,
      0.004434230873452432,
      0.9971666666666666,
      0.9976604278074866
    ]
  },
  "ShuffleNet_V1_GDConv": {
    "TAR": [
      0.19073212146759033,
      0.3839922117439465,
      0.9946666666666667,
      0.8405748663101604
    ],
    "FAR": [
      0.19073212146759033,
      0.01115297027235573,
      0.9946666666666667,
      0.9941510695187166
    ]
  },
  "ResNet50-casia": {
    "TAR": [
      0.1854616403579712,
      0.4882652428374924,
      0.986,
      0.9002339572192514
    ],
    "FAR": [
      0.1854616403579712,
      0.00643784274680342,
      0.986,
      0.99298128342246
    ]
  },
  "IR50-Am": {
    "TAR": [
      0.45836973190307617,
      0.5704550902847261,
      0.9921666666666666,
      0.8188502673796791
    ],
    "FAR": [
      0.45836973190307617,
      0.31482831996063737,
      0.9921666666666666,
      0.9899732620320856
    ]
  },
  "IR50-ArcFace": {
    "TAR": [
      0.27737492322921753,
      0.5110677213923673,
      0.997,
      0.8704879679144385
    ],
    "FAR": [
      0.27737492322921753,
      0.003643383801055488,
      0.997,
      0.9979946524064172
    ]
  },
  "IR50-CosFace": {
    "TAR": [
      0.22401964664459229,
      0.46094690714084996,
      0.997,
      0.8808489304812834
    ],
    "FAR": [
      0.22401964664459229,
      0.005503477108086469,
      0.997,
      0.9983288770053476
    ]
  },
  "IR50-Softmax": {
    "TAR": [
      0.34233152866363525,
      0.5712125343809098,
      0.996,
      0.8648061497326203
    ],
    "FAR": [
      0.34233152866363525,
      0.0364527001526091,
      0.996,
      0.9934826203208557
    ]
  },
  "IR50-SphereFace": {
    "TAR": [
      0.361619234085083,
      0.5931625330537734,
      0.9958333333333333,
      0.8724933155080213
    ],
    "FAR": [
      0.361619234085083,
      0.021256128760826037,
      0.9958333333333333,
      0.99298128342246
    ]
  },
  "IR50-PGDSoftmax": {
    "TAR": [
      0.3052944242954254,
      0.3509319506831969,
      0.913,
      0.5599933155080213
    ],
    "FAR": [
      0.3052944242954254,
      0.09187880892187193,
      0.913,
      0.9680815508021391
    ]
  },
  "IR50-TradesSoftmax": {
    "TAR": [
      0.32782697677612305,
      0.3710630280098767,
      0.9095,
      0.553475935828877
    ],
    "FAR": [
      0.32782697677612305,
      0.10951530974440651,
      0.9095,
      0.9608957219251337
    ]
  },
  "IR50-PGDArcFace": {
    "TAR": [
      0.37509775161743164,
      0.370836918276749,
      0.8766666666666667,
      0.4617312834224599
    ],
    "FAR": [
      0.37509775161743164,
      0.16079737244880937,
      0.8766666666666667,
      0.9336564171122995
    ]
  },
  "IR50-TradesArcFace": {
    "TAR": [
      0.852626383304596,
      0.8921198422975719,
      0.9506666666666667,
      0.8038101604278075
    ],
    "FAR": [
      0.852626383304596,
      0.7322988682501217,
      0.9506666666666667,
      0.914605614973262
    ]
  },
  "IR50-PGDCosFace": {
    "TAR": [
      0.2336357682943344,
      0.25343716062993227,
      0.8618333333333333,
      0.46858288770053474
    ],
    "FAR": [
      0.2336357682943344,
      0.09355347970528448,
      0.8618333333333333,
      0.93048128342246
    ]
  },
  "IR50-TradesCosFace": {
    "TAR": [
      0.6367585062980652,
      0.6468343956285171,
      0.9073333333333333,
      0.5842245989304813
    ],
    "FAR": [
      0.6367585062980652,
      0.41287724130977144,
      0.9073333333333333,
      0.9268048128342246
    ]
  },
  "IR50-PGDAm": {
    "TAR": [
      0.415134072303772,
      0.4146073014116821,
      0.85,
      0.4405080213903743
    ],
    "FAR": [
      0.415134072303772,
      0.2796455072562763,
      0.85,
      0.9254679144385026
    ]
  },
  "IR50-PGDSphereFace": {
    "TAR": [
      0.5209366679191589,
      0.5811544464018481,
      0.8881666666666667,
      0.6475601604278075
    ],
    "FAR": [
      0.5209366679191589,
      0.04542567590362066,
      0.8881666666666667,
      0.9055815508021391
    ]
  },
  "CASIA-Am": {
    "TAR": [
      0.4614875912666321,
      0.5446289519749502,
      0.978,
      0.7369652406417112
    ],
    "FAR": [
      0.4614875912666321,
      0.2900612007877615,
      0.978,
      0.9933155080213903
    ]
  },
  "CASIA-ArcFace": {
    "TAR": [
      0.3242388069629669,
      0.5455540836874936,
      0.9905,
      0.8811831550802139
    ],
    "FAR": [
      0.3242388069629669,
      0.004610521347708016,
      0.9905,
      0.995822192513369
    ]
  },
  "CASIA-CosFace": {
    "TAR": [
      0.23147112131118774,
      0.4875467206836419,
      0.9911666666666666,
      0.9126002673796791
    ],
    "FAR": [
      0.23147112131118774,
      0.0042326239056814895,
      0.9911666666666666,
      0.9968248663101604
    ]
  },
  "CASIA-Softmax": {
    "TAR": [
      0.3241730332374573,
      0.5245274235506108,
      0.9883333333333333,
      0.8716577540106952
    ],
    "FAR": [
      0.3241730332374573,
      0.014037041079987869,
      0.9883333333333333,
      0.9938168449197861
    ]
  },
  "CASIA-SphereFace": {
    "TAR": [
      0.4365990459918976,
      0.6730080748601465,
      0.9765,
      0.8805147058823529
    ],
    "FAR": [
      0.4365990459918976,
      0.0007960228080177097,
      0.9765,
      0.9886363636363636
    ]
  },
  "IR50-Softmax-BR": {
    "TAR": [
      0.33871138095855713,
      0.564035907398628,
      0.996,
      0.8628008021390374
    ],
    "FAR": [
      0.33871138095855713,
      0.03576667112213333,
      0.996,
      0.9956550802139037
    ]
  },
  "IR50-Softmax-JPEG": {
    "TAR": [
      0.34150177240371704,
      0.5678213814094962,
      0.9958333333333333,
      0.8624665775401069
    ],
    "FAR": [
      0.34150177240371704,
      0.0390969421559687,
      0.9958333333333333,
      0.9941510695187166
    ]
  },
  "IR50-Softmax-RP": {
    "TAR": [
      0.35351189970970154,
      0.5682759490413504,
      0.9941666666666666,
      0.8571189839572193
    ],
    "FAR": [
      0.35351189970970154,
      0.05020130523418999,
      0.9941666666666666,
      0.9936497326203209
    ]
  }
}
'''

455594 images of 10575 classes loaded
455594 images of 10575 classes loaded
使用数据集 /home/com/yy/datasets/CASIA-WebFace/casia-aligned-112x112/:
评估ShuffleNet_V2_GDConv-stride1中:
Load existing checkpoint
    TAR:


100%|██████████| 187/187 [00:48<00:00,  3.85it/s]


识别阈值: 0.22403636574745178
平均相似分数: 0.38950889972352953
ShuffleNet_V2_GDConv-stride1 benchmark rate:0.991500
ShuffleNet_V2_GDConv-stride1 success rate:0.807988
    FAR:


100%|██████████| 187/187 [01:47<00:00,  1.74it/s]


识别阈值: 0.22403636574745178
平均相似分数: 0.03514139656262923
ShuffleNet_V2_GDConv-stride1 benchmark rate:0.991500
ShuffleNet_V2_GDConv-stride1 success rate:0.992981
评估Mobilenet中:
Load existing checkpoint
    TAR:


...

识别阈值: 0.34150177240371704
平均相似分数: 0.0390969421559687
IR50-Softmax-JPEG benchmark rate:0.995833
IR50-Softmax-JPEG success rate:0.994151
评估IR50-Softmax-RP中:
Load existing checkpoint
    TAR:


100%|██████████| 187/187 [03:45<00:00,  1.21s/it]


识别阈值: 0.35351189970970154
平均相似分数: 0.5682759490413504
IR50-Softmax-RP benchmark rate:0.994167
IR50-Softmax-RP success rate:0.857119
    FAR:


100%|██████████| 187/187 [03:50<00:00,  1.23s/it]

识别阈值: 0.35351189970970154
平均相似分数: 0.05020130523418999
IR50-Softmax-RP benchmark rate:0.994167
IR50-Softmax-RP success rate:0.993650
{
  "ShuffleNet_V2_GDConv-stride1": {
    "TAR": [
      0.22403636574745178,
      0.38950889972352953,
      0.9915,
      0.8079879679144385
    ],
    "FAR": [
      0.22403636574745178,
      0.03514139656262923,
      0.9915,
      0.99298128342246
    ]
  },
  "Mobilenet": {
    "TAR": [
      0.15846577286720276,
      0.33970919899009966,
      0.9935,
      0.8153409090909091
    ],
    "FAR": [
      0.15846577286720276,
      0.007424565412303032,
      0.9935,
      0.9944852941176471
    ]
  },
  "MobilenetV2-stride1": {
    "TAR": [
      0.24340271949768066,
      0.4057902479143521,
      0.9905,
      0.797292780748663
    ],
    "FAR": [
      0.24340271949768066,
      0.03793284978065764,
      0.9905,
      0.9914772727272727
    ]
  },
  "CosFace": {
    "TAR": [
      0.24625787138938904,
      0.4757590359064407,
      0.9868333333